# Question 1

In [1]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/836809378.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Create the optimization model
model = gb.Model("Question 1: Canola Oil")

Restricted license - for non-production use only - expires 2025-11-24


In [3]:
# Read costs from CSV files
direct_shipping_costs_df = pd.read_csv('/Users/pratiksha/Documents/Schulich/Models and Applications in Operational Research/Cost_Production_to_Refinement.csv')
shipping_to_transsipment_costs_df = pd.read_csv('/Users/pratiksha/Documents/Schulich/Models and Applications in Operational Research/Cost_Production_to_Transshipment.csv')
transsipment_to_refinement_costs_df = pd.read_csv('/Users/pratiksha/Documents/Schulich/Models and Applications in Operational Research/Cost_Transshipment_to_Refinement.csv')

In [4]:
# Extract the "Cost" column
direct_shipping_costs = direct_shipping_costs_df['Cost'].values.reshape(25, -1)
shipping_to_transsipment_costs = shipping_to_transsipment_costs_df['Cost'].values.reshape(15, -1)
transsipment_to_refinement_costs = transsipment_to_refinement_costs_df['Cost'].values.reshape(2, -1)

In [5]:
direct_shipping_costs

array([[4.25273277, 4.56772552, 4.69648446, 2.67874096, 4.27245069],
       [2.95047222, 2.63661931, 2.85346871, 5.48002613, 3.03749539],
       [4.02568313, 5.40779944, 3.38094894, 5.44123713, 4.40886217],
       [3.46114107, 5.34694389, 4.99366311, 5.96818684, 4.83600407],
       [5.94681429, 4.74917249, 3.72614981, 3.58915858, 4.93741459],
       [3.46939366, 2.75501443, 5.60123055, 4.49910112, 4.22405746],
       [4.7290315 , 4.30193766, 3.67740309, 4.15881121, 4.15956806],
       [5.74380976, 5.57158666, 5.09534814, 5.34986497, 2.62562966],
       [4.25622982, 5.56865574, 4.13714572, 4.90410382, 3.05688139],
       [4.29664067, 5.75680178, 4.87474585, 4.10968275, 4.66040916],
       [4.02164417, 5.38745866, 5.4089004 , 5.18510303, 3.47259321],
       [3.28957805, 3.58831517, 3.62182771, 4.84919596, 4.89636402],
       [2.9072415 , 4.15142098, 4.60651418, 5.22233913, 3.76568815],
       [5.80493436, 3.58642269, 4.32677984, 5.63769746, 3.39212168],
       [5.87454415, 3.572913  , 3.

In [6]:
shipping_to_transsipment_costs

array([[2.37882586, 0.86384215],
       [1.66698156, 2.11948817],
       [2.17488041, 0.94818376],
       [2.67741553, 0.5924201 ],
       [2.82024783, 0.97232892],
       [2.85389005, 2.6943427 ],
       [1.40702817, 1.42832532],
       [2.48499907, 0.95347666],
       [1.78637201, 0.63669962],
       [0.68852771, 2.25970328],
       [2.40128579, 1.80010665],
       [0.56435019, 2.12484244],
       [1.98606958, 2.34717953],
       [1.63996858, 1.61928103],
       [0.64904663, 1.96881166]])

In [7]:
transsipment_to_refinement_costs

array([[1.57232851, 3.46547396, 2.24406225, 3.77383915, 3.2626518 ],
       [1.90042452, 1.6131232 , 1.70723241, 2.70539568, 2.25742842]])

In [8]:
# Create the a single class of decision variables where
# From = {Ca,US,M,C,F} and To = {R1,R2,R3,R4,R5}.
x = model.addVars(25, 5, lb=0, vtype=GRB.CONTINUOUS, name="Direct Shipping")
# From = {I,U,G} and To = {Italy,Greece}.
y = model.addVars(15, 2, lb=0, vtype=GRB.CONTINUOUS, name="Ship to Transshipment")
# From = {Italy,Greece} and To = {R1,R2,R3,R4,R5}.
z = model.addVars(2, 5, lb=0, vtype=GRB.CONTINUOUS, name="Transshipment to Refinement")

In [9]:
# The objective function
direct_objective = gb.quicksum(direct_shipping_costs[i][j]*x[i,j] for i in range(25) for j in range(5))
trans_objective = gb.quicksum(shipping_to_transsipment_costs[i][j]*y[i,j] for i in range(15) for j in range(2))
trans_to_refinement_objective = gb.quicksum(transsipment_to_refinement_costs[i][j]*z[i,j] for i in range(2) for j in range(5))
model.setObjective(direct_objective + trans_objective + trans_to_refinement_objective, GRB.MINIMIZE)

In [10]:
# Read capacity and demand from CSV files
direct_shipping_supply_capacity_df = pd.read_csv('/Users/pratiksha/Documents/Schulich/Models and Applications in Operational Research/Capacity_for_Direct_Production_Facilities.csv')
transshipment_supply_capacity_df = pd.read_csv('/Users/pratiksha/Documents/Schulich/Models and Applications in Operational Research/Capacity_for_Transship_Distribution_Centers.csv')
shipping_to_transshipment_supply_capacity_df = pd.read_csv('/Users/pratiksha/Documents/Schulich/Models and Applications in Operational Research/Capacity_for_Transship_Production_Facilities.csv')
refinement_demand_df = pd.read_csv('/Users/pratiksha/Documents/Schulich/Models and Applications in Operational Research/Refinement_Demand.csv')

In [11]:
# Extract the "Capacity" and "Demand" column
direct_shipping_supply_capacity = direct_shipping_supply_capacity_df['Capacity'].values.reshape(25, -1)
transshipment_supply_capacity = transshipment_supply_capacity_df['Capacity'].values.reshape(2, -1)
shipping_to_transshipment_supply_capacity = shipping_to_transshipment_supply_capacity_df['Capacity'].values.reshape(15, -1)
refinement_demand = refinement_demand_df['Demand'].values.reshape(5, -1)

In [12]:
direct_shipping_supply_capacity

array([[462],
       [103],
       [460],
       [325],
       [227],
       [217],
       [205],
       [521],
       [548],
       [191],
       [361],
       [411],
       [104],
       [155],
       [285],
       [109],
       [422],
       [438],
       [501],
       [139],
       [462],
       [504],
       [106],
       [132],
       [298]])

In [13]:
transshipment_supply_capacity

array([[1317],
       [1453]])

In [14]:
shipping_to_transshipment_supply_capacity

array([[374],
       [444],
       [395],
       [245],
       [378],
       [408],
       [435],
       [175],
       [415],
       [503],
       [184],
       [297],
       [450],
       [169],
       [365]])

In [15]:
refinement_demand

array([[1537],
       [1748],
       [1940],
       [1838],
       [1665]])

In [16]:
# Add the supply constraints from source nodes for direct shipping
for i in range(len(direct_shipping_supply_capacity)):
    model.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= direct_shipping_supply_capacity[i], name=f"Direct Supply Constraint {i + 1}")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/29224195.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= direct_shipping_supply_capacity[i], name=f"Direct Supply Constraint {i + 1}")


In [17]:
# Add the supply constraints from source nodes for transshipment shipping
for i in range(len(shipping_to_transshipment_supply_capacity)):
    model.addConstr(gb.quicksum(y[i, j] for j in range(2)) <= shipping_to_transshipment_supply_capacity[i], name=f"Transshipment Supply Constraint {i + 1}")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/1905600687.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model.addConstr(gb.quicksum(y[i, j] for j in range(2)) <= shipping_to_transshipment_supply_capacity[i], name=f"Transshipment Supply Constraint {i + 1}")


In [18]:
# Add the supply constraints from transshipment nodes
model.addConstr(gb.quicksum(y[i,0] for i in range(15)) <= transshipment_supply_capacity[0], name="Transship Capacity 1")
model.addConstr(gb.quicksum(y[i,1] for i in range(15)) <= transshipment_supply_capacity[1], name="Transship Capacity 2")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/2124321848.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model.addConstr(gb.quicksum(y[i,0] for i in range(15)) <= transshipment_supply_capacity[0], name="Transship Capacity 1")
/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/2124321848.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model.addConstr(gb.quicksum(y[i,1] for i in range(15)) <= transshipment_supply_capacity[1], name="Transship Capacity 2")


<gurobi.Constr *Awaiting Model Update*>

In [19]:
# Add the flow balance constrainits
model.addConstr(gb.quicksum(y[i,0] for i in range(15)) == gb.quicksum(z[0,k] for k in range(5)), name="Flow Balance 1")
model.addConstr(gb.quicksum(y[i,1] for i in range(15)) == gb.quicksum(z[1,k] for k in range(5)), name="Flow Balance 2")

<gurobi.Constr *Awaiting Model Update*>

In [20]:
# Add the demand constraints
for k in range(len(refinement_demand)):  # Iterate over refinement nodes R1 to R5
    model.addConstr(gb.quicksum(x[i, k] for i in range(25)) + gb.quicksum(z[j, k] for j in range(2)) == refinement_demand[k], name=f"Refinement Demand Constraint {k + 1}")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/4255670903.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model.addConstr(gb.quicksum(x[i, k] for i in range(25)) + gb.quicksum(z[j, k] for j in range(2)) == refinement_demand[k], name=f"Refinement Demand Constraint {k + 1}")


In [21]:
# Optimally solve the problem
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 49 rows, 165 columns and 360 nonzeros
Model fingerprint: 0x7f8fadb4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.00s
Presolved: 49 rows, 165 columns, 360 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7230583e+04   8.266000e+03   0.000000e+00      0s
      36    2.4188585e+04   0.000000e+00   0.000000e+00      0s

Solved in 36 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.418858517e+04


In [22]:
# The status of the model (Optimization Status Codes)
print("Model Status: ", model.status)

Model Status:  2


In [23]:
# Number of variables in the model
print("Number of Decision Variables: ", model.numVars)

Number of Decision Variables:  165


The answer to a)

In [24]:
# Value of the objective function
print("Total Transportation cost: ", round(model.objVal, 2))

Total Transportation cost:  24188.59


In [25]:
# Print the decision variables
print(model.printAttr('X'))


    Variable            X 
-------------------------
Direct Shipping[0,3]          462 
Direct Shipping[1,1]          103 
Direct Shipping[2,2]          460 
Direct Shipping[4,3]           86 
Direct Shipping[5,1]          217 
Direct Shipping[7,4]          521 
Direct Shipping[8,4]          548 
Direct Shipping[10,4]          354 
Direct Shipping[11,0]            7 
Direct Shipping[11,2]          404 
Direct Shipping[12,0]          104 
Direct Shipping[13,4]          155 
Direct Shipping[14,3]          285 
Direct Shipping[15,0]          109 
Direct Shipping[17,1]          351 
Direct Shipping[17,4]           87 
Direct Shipping[18,3]          501 
Direct Shipping[20,2]          462 
Direct Shipping[21,3]          504 
Direct Shipping[22,1]          106 
Direct Shipping[23,2]          132 
Ship to Transshipment[0,1]          374 
Ship to Transshipment[2,1]          395 
Ship to Transshipment[3,1]          245 
Ship to Transshipment[6,0]          152 
Ship to Transshipment[7,1]       

In [26]:
# Get the optimal values of decision variables
to_transshipment_values = model.getAttr('x', y)
to_transshipment_values

{(0, 0): 0.0,
 (0, 1): 374.0,
 (1, 0): 0.0,
 (1, 1): 0.0,
 (2, 0): 0.0,
 (2, 1): 395.0,
 (3, 0): 0.0,
 (3, 1): 245.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (5, 0): 0.0,
 (5, 1): 0.0,
 (6, 0): 152.0,
 (6, 1): 0.0,
 (7, 0): 0.0,
 (7, 1): 24.0,
 (8, 0): 0.0,
 (8, 1): 415.0,
 (9, 0): 503.0,
 (9, 1): 0.0,
 (10, 0): 0.0,
 (10, 1): 0.0,
 (11, 0): 297.0,
 (11, 1): 0.0,
 (12, 0): 0.0,
 (12, 1): 0.0,
 (13, 0): 0.0,
 (13, 1): 0.0,
 (14, 0): 365.0,
 (14, 1): 0.0}

In [27]:
values_array = np.array(list(to_transshipment_values.values()))

In [28]:
# Calculate the mean
mean_value = np.mean(values_array)
mean_value

92.33333333333333

In [29]:
# Get the optimal values of decision variables
transshipment_values = model.getAttr('x', z)
transshipment_values

{(0, 0): 1317.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 971.0,
 (1, 2): 482.0,
 (1, 3): 0.0,
 (1, 4): 0.0}

In [30]:
# Calculate the total amount of canola oil transshipped
total_transshipped = sum(transshipment_values[i, j] for i in range(2) for j in range(5))
total_transshipped

2770.0

In [31]:
# Get the optimal values of decision variables
direct_shipping_values = model.getAttr('x', x)
direct_shipping_values

{(0, 0): 0.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 462.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 103.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 460.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 0.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 86.0,
 (4, 4): 0.0,
 (5, 0): 0.0,
 (5, 1): 217.0,
 (5, 2): 0.0,
 (5, 3): 0.0,
 (5, 4): 0.0,
 (6, 0): 0.0,
 (6, 1): 0.0,
 (6, 2): 0.0,
 (6, 3): 0.0,
 (6, 4): 0.0,
 (7, 0): 0.0,
 (7, 1): 0.0,
 (7, 2): 0.0,
 (7, 3): 0.0,
 (7, 4): 521.0,
 (8, 0): 0.0,
 (8, 1): 0.0,
 (8, 2): 0.0,
 (8, 3): 0.0,
 (8, 4): 548.0,
 (9, 0): 0.0,
 (9, 1): 0.0,
 (9, 2): 0.0,
 (9, 3): 0.0,
 (9, 4): 0.0,
 (10, 0): 0.0,
 (10, 1): 0.0,
 (10, 2): 0.0,
 (10, 3): 0.0,
 (10, 4): 354.0,
 (11, 0): 7.0,
 (11, 1): 0.0,
 (11, 2): 404.0,
 (11, 3): 0.0,
 (11, 4): 0.0,
 (12, 0): 104.0,
 (12, 1): 0.0,
 (12, 2): 0.0,
 (12, 3): 0.0,
 (12, 4): 0.0,
 (13, 0): 0.0,
 (13, 1): 0.0,
 (13, 2): 0.0,
 (13, 3): 0

In [32]:
# Calculate the total amount of canola oil directly shipped
direct_shipping = sum(direct_shipping_values[i, j] for i in range(25) for j in range(5))
direct_shipping

5958.0

In [33]:
# Calculate the total amount of canola oil directly shipped for North America
direct_shipping_for_f = sum(direct_shipping_values[i, j] for i in range(15) for j in range(5))
direct_shipping_for_f

3706.0

The answer to b)

In [34]:
# Calculate the proportion
total_canola_oil = direct_shipping + total_transshipped
proportion_transshipped = total_transshipped / total_canola_oil
proportion_transshipped

0.31736938588450964

In [35]:
# Create the optimization model
model_modified = gb.Model("Question 1: Canola Oil Objective Function Change")

In [36]:
# Create the a single class of decision variables where
# From = {Ca,US,M,C,F} and To = {R1,R2,R3,R4,R5}.
x = model_modified.addVars(25, 5, lb=0, vtype=GRB.CONTINUOUS, name="Direct Shipping")
# From = {I,U,G} and To = {Italy,Greece}.
y = model_modified.addVars(15, 2, lb=0, vtype=GRB.CONTINUOUS, name="Ship to Transshipment")
# From = {Italy,Greece} and To = {R1,R2,R3,R4,R5}.
z = model_modified.addVars(2, 5, lb=0, vtype=GRB.CONTINUOUS, name="Transshipment to Refinement")

In [37]:
direct_shipping_costs

array([[4.25273277, 4.56772552, 4.69648446, 2.67874096, 4.27245069],
       [2.95047222, 2.63661931, 2.85346871, 5.48002613, 3.03749539],
       [4.02568313, 5.40779944, 3.38094894, 5.44123713, 4.40886217],
       [3.46114107, 5.34694389, 4.99366311, 5.96818684, 4.83600407],
       [5.94681429, 4.74917249, 3.72614981, 3.58915858, 4.93741459],
       [3.46939366, 2.75501443, 5.60123055, 4.49910112, 4.22405746],
       [4.7290315 , 4.30193766, 3.67740309, 4.15881121, 4.15956806],
       [5.74380976, 5.57158666, 5.09534814, 5.34986497, 2.62562966],
       [4.25622982, 5.56865574, 4.13714572, 4.90410382, 3.05688139],
       [4.29664067, 5.75680178, 4.87474585, 4.10968275, 4.66040916],
       [4.02164417, 5.38745866, 5.4089004 , 5.18510303, 3.47259321],
       [3.28957805, 3.58831517, 3.62182771, 4.84919596, 4.89636402],
       [2.9072415 , 4.15142098, 4.60651418, 5.22233913, 3.76568815],
       [5.80493436, 3.58642269, 4.32677984, 5.63769746, 3.39212168],
       [5.87454415, 3.572913  , 3.

In [38]:
# The objective function
direct_objective = gb.quicksum(direct_shipping_costs[i][j]*x[i,j] for i in range(25) for j in range(5))
trans_objective = gb.quicksum(shipping_to_transsipment_costs[i][j]*y[i,j] for i in range(15) for j in range(2))
trans_to_refinement_objective = gb.quicksum(transsipment_to_refinement_costs[i][j]*z[i,j] for i in range(2) for j in range(5))

# Adding the modified objective to the model
model_modified.setObjective((0.95*direct_objective) + (1.15*trans_objective) + trans_to_refinement_objective, GRB.MINIMIZE)

In [39]:
# Add the supply constraints from source nodes for direct shipping
for i in range(len(direct_shipping_supply_capacity)):
    model_modified.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= direct_shipping_supply_capacity[i], name=f"Direct Supply Constraint {i + 1}")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/444835003.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model_modified.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= direct_shipping_supply_capacity[i], name=f"Direct Supply Constraint {i + 1}")


In [40]:
# Add the supply constraints from source nodes for transshipment shipping
for i in range(len(shipping_to_transshipment_supply_capacity)):
    model_modified.addConstr(gb.quicksum(y[i, j] for j in range(2)) <= shipping_to_transshipment_supply_capacity[i], name=f"Transshipment Supply Constraint {i + 1}")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/334111189.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model_modified.addConstr(gb.quicksum(y[i, j] for j in range(2)) <= shipping_to_transshipment_supply_capacity[i], name=f"Transshipment Supply Constraint {i + 1}")


In [41]:
# Add the supply constraints from transshipment nodes
model_modified.addConstr(gb.quicksum(y[i,0] for i in range(15)) <= transshipment_supply_capacity[0], name="Transship Capacity 1")
model_modified.addConstr(gb.quicksum(y[i,1] for i in range(15)) <= transshipment_supply_capacity[1], name="Transship Capacity 2")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/1084066422.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model_modified.addConstr(gb.quicksum(y[i,0] for i in range(15)) <= transshipment_supply_capacity[0], name="Transship Capacity 1")
/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/1084066422.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model_modified.addConstr(gb.quicksum(y[i,1] for i in range(15)) <= transshipment_supply_capacity[1], name="Transship Capacity 2")


<gurobi.Constr *Awaiting Model Update*>

In [42]:
# Add the flow balance constrainits
model_modified.addConstr(gb.quicksum(y[i,0] for i in range(15)) == gb.quicksum(z[0,k] for k in range(5)), name="Flow Balance 1")
model_modified.addConstr(gb.quicksum(y[i,1] for i in range(15)) == gb.quicksum(z[1,k] for k in range(5)), name="Flow Balance 2")

<gurobi.Constr *Awaiting Model Update*>

In [43]:
# Add the demand constraints
for k in range(len(refinement_demand)):  # Iterate over refinement nodes R1 to R5
    model_modified.addConstr(gb.quicksum(x[i, k] for i in range(25)) + gb.quicksum(z[j, k] for j in range(2)) == refinement_demand[k], name=f"Refinement Demand Constraint {k + 1}")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/1335377250.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model_modified.addConstr(gb.quicksum(x[i, k] for i in range(25)) + gb.quicksum(z[j, k] for j in range(2)) == refinement_demand[k], name=f"Refinement Demand Constraint {k + 1}")


In [44]:
# Optimally solve the problem
model_modified.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 49 rows, 165 columns and 360 nonzeros
Model fingerprint: 0x8c9b5881
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.00s
Presolved: 49 rows, 165 columns, 360 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6984407e+04   8.266000e+03   0.000000e+00      0s
      35    2.3619036e+04   0.000000e+00   0.000000e+00      0s

Solved in 35 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.361903559e+04


In [45]:
# Value of the objective function
print("Total Transportation cost: ", round(model_modified.objVal, 2))

Total Transportation cost:  23619.04


In [46]:
# Get the optimal values of decision variables
transshipment_values_modified = model_modified.getAttr('x', y)
transshipment_values_modified

{(0, 0): 0.0,
 (0, 1): 374.0,
 (1, 0): 0.0,
 (1, 1): 0.0,
 (2, 0): 0.0,
 (2, 1): 395.0,
 (3, 0): 0.0,
 (3, 1): 245.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (5, 0): 0.0,
 (5, 1): 0.0,
 (6, 0): 0.0,
 (6, 1): 0.0,
 (7, 0): 0.0,
 (7, 1): 24.0,
 (8, 0): 0.0,
 (8, 1): 415.0,
 (9, 0): 503.0,
 (9, 1): 0.0,
 (10, 0): 0.0,
 (10, 1): 0.0,
 (11, 0): 297.0,
 (11, 1): 0.0,
 (12, 0): 0.0,
 (12, 1): 0.0,
 (13, 0): 0.0,
 (13, 1): 0.0,
 (14, 0): 365.0,
 (14, 1): 0.0}

In [47]:
# Calculate the total amount of canola oil transshipped
total_transshipped_modified = sum(transshipment_values_modified[i, j] for i in range(15) for j in range(2))
total_transshipped_modified

2618.0

Part d)

In [48]:
# Create the optimization model
model_constraint = gb.Model("Question 1: Canola Oil Constraint Change")

In [49]:
# Create the a single class of decision variables where
# From = {Ca,US,M,C,F} and To = {R1,R2,R3,R4,R5}.
x = model_constraint.addVars(25, 5, lb=0, vtype=GRB.CONTINUOUS, name="Direct Shipping")
# From = {I,U,G} and To = {Italy,Greece}.
y = model_constraint.addVars(15, 2, lb=0, vtype=GRB.CONTINUOUS, name="Ship to Transshipment")
# From = {Italy,Greece} and To = {R1,R2,R3,R4,R5}.
z = model_constraint.addVars(2, 5, lb=0, vtype=GRB.CONTINUOUS, name="Transshipment to Refinement")

In [50]:
# The objective function
direct_objective = gb.quicksum(direct_shipping_costs[i][j]*x[i,j] for i in range(25) for j in range(5))
trans_objective = gb.quicksum(shipping_to_transsipment_costs[i][j]*y[i,j] for i in range(15) for j in range(2))
trans_to_refinement_objective = gb.quicksum(transsipment_to_refinement_costs[i][j]*z[i,j] for i in range(2) for j in range(5))
model_constraint.setObjective(direct_objective + trans_objective + trans_to_refinement_objective, GRB.MINIMIZE)

In [51]:
# Add the supply constraints from source nodes for direct shipping
for i in range(len(direct_shipping_supply_capacity)):
    model_constraint.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= direct_shipping_supply_capacity[i], name=f"Direct Supply Constraint {i + 1}")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/1810285693.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model_constraint.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= direct_shipping_supply_capacity[i], name=f"Direct Supply Constraint {i + 1}")


In [52]:
# Add the supply constraints from source nodes for transshipment shipping
for i in range(len(shipping_to_transshipment_supply_capacity)):
    model_constraint.addConstr(gb.quicksum(y[i, j] for j in range(2)) <= shipping_to_transshipment_supply_capacity[i], name=f"Transshipment Supply Constraint {i + 1}")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/683186048.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model_constraint.addConstr(gb.quicksum(y[i, j] for j in range(2)) <= shipping_to_transshipment_supply_capacity[i], name=f"Transshipment Supply Constraint {i + 1}")


In [53]:
# Add the supply constraints from transshipment nodes
model_constraint.addConstr(gb.quicksum(y[i,0] for i in range(15)) <= transshipment_supply_capacity[0], name="Transship Capacity 1")
model_constraint.addConstr(gb.quicksum(y[i,1] for i in range(15)) <= transshipment_supply_capacity[1], name="Transship Capacity 2")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/2032879190.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model_constraint.addConstr(gb.quicksum(y[i,0] for i in range(15)) <= transshipment_supply_capacity[0], name="Transship Capacity 1")
/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/2032879190.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model_constraint.addConstr(gb.quicksum(y[i,1] for i in range(15)) <= transshipment_supply_capacity[1], name="Transship Capacity 2")


<gurobi.Constr *Awaiting Model Update*>

In [54]:
# Add the flow balance constrainits
model_constraint.addConstr(gb.quicksum(y[i,0] for i in range(15)) == gb.quicksum(z[0,k] for k in range(5)), name="Flow Balance 1")
model_constraint.addConstr(gb.quicksum(y[i,1] for i in range(15)) == gb.quicksum(z[1,k] for k in range(5)), name="Flow Balance 2")

<gurobi.Constr *Awaiting Model Update*>

In [55]:
# Add the demand constraints
for k in range(len(refinement_demand)):  # Iterate over refinement nodes R1 to R5
    model_constraint.addConstr(gb.quicksum(x[i, k] for i in range(25)) + gb.quicksum(z[j, k] for j in range(2)) == refinement_demand[k], name=f"Refinement Demand Constraint {k + 1}")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/3499786536.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model_constraint.addConstr(gb.quicksum(x[i, k] for i in range(25)) + gb.quicksum(z[j, k] for j in range(2)) == refinement_demand[k], name=f"Refinement Demand Constraint {k + 1}")


Added constraint

In [56]:
# Ratio constraint
model_constraint.addConstr((0.25*(gb.quicksum(x[i,j] for i in range(25) for j in range(5)) + gb.quicksum(z[i,j] for i in range(2) for j in range(5)))) >= gb.quicksum(y[i,j] for i in range(15) for j in range(2)), name="Ratio constraint")

<gurobi.Constr *Awaiting Model Update*>

In [57]:
# Optimally solve the problem
model_constraint.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50 rows, 165 columns and 525 nonzeros
Model fingerprint: 0x5628116b
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.00s
Presolved: 50 rows, 165 columns, 525 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7230583e+04   8.266000e+03   0.000000e+00      0s
      35    2.4720228e+04   0.000000e+00   0.000000e+00      0s

Solved in 35 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.472022754e+04


In [58]:
# Value of the objective function
print("Total Transportation cost: ", round(model_constraint.objVal, 2))

Total Transportation cost:  24720.23


In [59]:
# Get the optimal values of decision variables
transshipment_values = model_constraint.getAttr('x', z)
transshipment_values

{(0, 0): 1148.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 964.0,
 (1, 2): 70.0,
 (1, 3): 0.0,
 (1, 4): 0.0}

In [60]:
# Calculate the total amount of canola oil transshipped
total_transshipped = sum(transshipment_values[i, j] for i in range(2) for j in range(5))
total_transshipped

2182.0

In [61]:
# Get the optimal values of decision variables
direct_shipping_values = model_constraint.getAttr('x', x)
direct_shipping_values

{(0, 0): 0.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 462.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 103.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 460.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 149.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 0.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 227.0,
 (4, 3): 0.0,
 (4, 4): 0.0,
 (5, 0): 0.0,
 (5, 1): 217.0,
 (5, 2): 0.0,
 (5, 3): 0.0,
 (5, 4): 0.0,
 (6, 0): 0.0,
 (6, 1): 0.0,
 (6, 2): 205.0,
 (6, 3): 0.0,
 (6, 4): 0.0,
 (7, 0): 0.0,
 (7, 1): 0.0,
 (7, 2): 0.0,
 (7, 3): 0.0,
 (7, 4): 521.0,
 (8, 0): 0.0,
 (8, 1): 0.0,
 (8, 2): 0.0,
 (8, 3): 0.0,
 (8, 4): 548.0,
 (9, 0): 0.0,
 (9, 1): 0.0,
 (9, 2): 0.0,
 (9, 3): 0.0,
 (9, 4): 0.0,
 (10, 0): 0.0,
 (10, 1): 0.0,
 (10, 2): 0.0,
 (10, 3): 0.0,
 (10, 4): 361.0,
 (11, 0): 27.0,
 (11, 1): 0.0,
 (11, 2): 384.0,
 (11, 3): 0.0,
 (11, 4): 0.0,
 (12, 0): 104.0,
 (12, 1): 0.0,
 (12, 2): 0.0,
 (12, 3): 0.0,
 (12, 4): 0.0,
 (13, 0): 0.0,
 (13, 1): 0.0,
 (13, 2): 0.0,
 (13,

In [62]:
# Calculate the total amount of canola oil directly shipped
direct_shipping = sum(direct_shipping_values[i, j] for i in range(25) for j in range(5))
direct_shipping

6546.0

In [63]:
# Calculate the proportion
total_canola_oil = direct_shipping + total_transshipped
proportion_transshipped = total_transshipped / total_canola_oil
proportion_transshipped

0.25

e) Choose the constraint route for obvious reasons

f)

In [64]:
# Create the optimization model
model_NA = gb.Model("Question 1: Canola Oil North America Change")

In [65]:
# Create the a single class of decision variables where
# From = {Ca,US,M,C,F} and To = {R1,R2,R3,R4,R5}.
x = model_NA.addVars(25, 5, lb=0, vtype=GRB.CONTINUOUS, name="Direct Shipping")
# From = {I,U,G} and To = {Italy,Greece}.
y = model_NA.addVars(15, 2, lb=0, vtype=GRB.CONTINUOUS, name="Ship to Transshipment")
# From = {Italy,Greece} and To = {R1,R2,R3,R4,R5}.
z = model_NA.addVars(2, 5, lb=0, vtype=GRB.CONTINUOUS, name="Transshipment to Refinement")

Apply Discount

In [66]:
direct_shipping_costs

array([[4.25273277, 4.56772552, 4.69648446, 2.67874096, 4.27245069],
       [2.95047222, 2.63661931, 2.85346871, 5.48002613, 3.03749539],
       [4.02568313, 5.40779944, 3.38094894, 5.44123713, 4.40886217],
       [3.46114107, 5.34694389, 4.99366311, 5.96818684, 4.83600407],
       [5.94681429, 4.74917249, 3.72614981, 3.58915858, 4.93741459],
       [3.46939366, 2.75501443, 5.60123055, 4.49910112, 4.22405746],
       [4.7290315 , 4.30193766, 3.67740309, 4.15881121, 4.15956806],
       [5.74380976, 5.57158666, 5.09534814, 5.34986497, 2.62562966],
       [4.25622982, 5.56865574, 4.13714572, 4.90410382, 3.05688139],
       [4.29664067, 5.75680178, 4.87474585, 4.10968275, 4.66040916],
       [4.02164417, 5.38745866, 5.4089004 , 5.18510303, 3.47259321],
       [3.28957805, 3.58831517, 3.62182771, 4.84919596, 4.89636402],
       [2.9072415 , 4.15142098, 4.60651418, 5.22233913, 3.76568815],
       [5.80493436, 3.58642269, 4.32677984, 5.63769746, 3.39212168],
       [5.87454415, 3.572913  , 3.

In [67]:
for i in range(15):
    for j in range(5):
        direct_shipping_costs[i, j] *= 0.82

In [68]:
direct_shipping_costs

array([[3.48724087, 3.74553493, 3.85111726, 2.19656759, 3.50340957],
       [2.41938722, 2.16202783, 2.33984434, 4.49362142, 2.49074622],
       [3.30106017, 4.43439554, 2.77237813, 4.46181444, 3.61526698],
       [2.83813568, 4.38449399, 4.09480375, 4.89391321, 3.96552334],
       [4.87638772, 3.89432144, 3.05544285, 2.94311004, 4.04867996],
       [2.8449028 , 2.25911183, 4.59300905, 3.68926292, 3.46372712],
       [3.87780583, 3.52758888, 3.01547053, 3.41022519, 3.41084581],
       [4.709924  , 4.56870106, 4.17818548, 4.38688928, 2.15301632],
       [3.49010845, 4.5662977 , 3.39245949, 4.02136514, 2.50664274],
       [3.52324535, 4.72057746, 3.9972916 , 3.36993986, 3.82153551],
       [3.29774822, 4.4177161 , 4.43529833, 4.25178449, 2.84752643],
       [2.697454  , 2.94241844, 2.96989873, 3.97634069, 4.0150185 ],
       [2.38393803, 3.40416521, 3.77734163, 4.28231809, 3.08786428],
       [4.76004618, 2.94086661, 3.54795947, 4.62291191, 2.78153978],
       [4.81712621, 2.92978866, 3.

In [69]:
# The objective function
direct_objective = gb.quicksum(direct_shipping_costs[i][j]*x[i,j] for i in range(25) for j in range(5))
trans_objective = gb.quicksum(shipping_to_transsipment_costs[i][j]*y[i,j] for i in range(15) for j in range(2))
trans_to_refinement_objective = gb.quicksum(transsipment_to_refinement_costs[i][j]*z[i,j] for i in range(2) for j in range(5))
model_NA.setObjective(direct_objective + trans_objective + trans_to_refinement_objective, GRB.MINIMIZE)

In [70]:
# Add the supply constraints from source nodes for direct shipping
for i in range(len(direct_shipping_supply_capacity)):
    model_NA.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= direct_shipping_supply_capacity[i], name=f"Direct Supply Constraint {i + 1}")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/2092599772.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model_NA.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= direct_shipping_supply_capacity[i], name=f"Direct Supply Constraint {i + 1}")


In [71]:
# Add the supply constraints from source nodes for transshipment shipping
for i in range(len(shipping_to_transshipment_supply_capacity)):
    model_NA.addConstr(gb.quicksum(y[i, j] for j in range(2)) <= shipping_to_transshipment_supply_capacity[i], name=f"Transshipment Supply Constraint {i + 1}")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/1085130033.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model_NA.addConstr(gb.quicksum(y[i, j] for j in range(2)) <= shipping_to_transshipment_supply_capacity[i], name=f"Transshipment Supply Constraint {i + 1}")


In [72]:
# Add the supply constraints from transshipment nodes
model_NA.addConstr(gb.quicksum(y[i,0] for i in range(15)) <= transshipment_supply_capacity[0], name="Transship Capacity 1")
model_NA.addConstr(gb.quicksum(y[i,1] for i in range(15)) <= transshipment_supply_capacity[1], name="Transship Capacity 2")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/375248322.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model_NA.addConstr(gb.quicksum(y[i,0] for i in range(15)) <= transshipment_supply_capacity[0], name="Transship Capacity 1")
/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/375248322.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model_NA.addConstr(gb.quicksum(y[i,1] for i in range(15)) <= transshipment_supply_capacity[1], name="Transship Capacity 2")


<gurobi.Constr *Awaiting Model Update*>

In [73]:
# Add the flow balance constrainits
model_NA.addConstr(gb.quicksum(y[i,0] for i in range(15)) == gb.quicksum(z[0,k] for k in range(5)), name="Flow Balance 1")
model_NA.addConstr(gb.quicksum(y[i,1] for i in range(15)) == gb.quicksum(z[1,k] for k in range(5)), name="Flow Balance 2")

<gurobi.Constr *Awaiting Model Update*>

In [74]:
# Add the demand constraints
for k in range(len(refinement_demand)):  # Iterate over refinement nodes R1 to R5
    model_NA.addConstr(gb.quicksum(x[i, k] for i in range(25)) + gb.quicksum(z[j, k] for j in range(2)) == refinement_demand[k], name=f"Refinement Demand Constraint {k + 1}")

/var/folders/9w/090_7gb54d54ppm4sfqk8xgr0000gn/T/ipykernel_18542/3413968090.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  model_NA.addConstr(gb.quicksum(x[i, k] for i in range(25)) + gb.quicksum(z[j, k] for j in range(2)) == refinement_demand[k], name=f"Refinement Demand Constraint {k + 1}")


In [75]:
# Optimally solve the problem
model_NA.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 49 rows, 165 columns and 360 nonzeros
Model fingerprint: 0x4654186c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.00s
Presolved: 49 rows, 165 columns, 360 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6170503e+04   7.745000e+03   0.000000e+00      0s
      31    2.2076176e+04   0.000000e+00   0.000000e+00      0s

Solved in 31 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.207617571e+04


In [76]:
# Value of the objective function
print("Total Transportation cost: ", round(model_NA.objVal, 2))

Total Transportation cost:  22076.18


In [77]:
# Get the optimal values of decision variables
direct_shipping_values = model_NA.getAttr('x', x)
direct_shipping_values

{(0, 0): 0.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 462.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 103.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 460.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 0.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 141.0,
 (4, 3): 86.0,
 (4, 4): 0.0,
 (5, 0): 0.0,
 (5, 1): 217.0,
 (5, 2): 0.0,
 (5, 3): 0.0,
 (5, 4): 0.0,
 (6, 0): 0.0,
 (6, 1): 0.0,
 (6, 2): 205.0,
 (6, 3): 0.0,
 (6, 4): 0.0,
 (7, 0): 0.0,
 (7, 1): 0.0,
 (7, 2): 0.0,
 (7, 3): 0.0,
 (7, 4): 521.0,
 (8, 0): 0.0,
 (8, 1): 0.0,
 (8, 2): 0.0,
 (8, 3): 0.0,
 (8, 4): 548.0,
 (9, 0): 0.0,
 (9, 1): 0.0,
 (9, 2): 0.0,
 (9, 3): 0.0,
 (9, 4): 0.0,
 (10, 0): 0.0,
 (10, 1): 0.0,
 (10, 2): 0.0,
 (10, 3): 0.0,
 (10, 4): 361.0,
 (11, 0): 228.0,
 (11, 1): 0.0,
 (11, 2): 183.0,
 (11, 3): 0.0,
 (11, 4): 0.0,
 (12, 0): 104.0,
 (12, 1): 0.0,
 (12, 2): 0.0,
 (12, 3): 0.0,
 (12, 4): 0.0,
 (13, 0): 0.0,
 (13, 1): 0.0,
 (13, 2): 0.0,
 (13,

In [78]:
# Calculate the total amount of canola oil directly shipped
direct_shipping = sum(direct_shipping_values[i, j] for i in range(25) for j in range(5))
direct_shipping

6110.0

In [79]:
# Calculate the total amount of canola oil directly shipped
direct_shipping_for_f = sum(direct_shipping_values[i, j] for i in range(15) for j in range(5))
direct_shipping_for_f

4059.0

# Question 2

In [80]:
# Create the optimization model
question_2_model = gb.Model("Question 2: Sunnyshore Bay")

In [81]:
# Create the three classes of decision variables where each Python
# variable represents a different number of Gurobi decision variables
B = question_2_model.addVars(6, lb=0, vtype=GRB.CONTINUOUS, name="Borrow")
w = question_2_model.addVars(4, lb=0, vtype=GRB.CONTINUOUS, name="Wealth")

In [82]:
# The objective function
question_2_model.setObjective(w[3], GRB.MAXIMIZE)

In [83]:
# Add the balance constraints
question_2_model.addConstr(w[0] == 140000 + 180000 - 300000 + B[0] + B[1] + B[2], "May Balance Constraint")
question_2_model.addConstr(w[1] == w[0] + 260000 - 400000 + B[3] + B[4] - 1.0175*B[2], "June Balance Constraint")
question_2_model.addConstr(w[2] == w[1] + 420000 - 350000 + B[5] - 1.0225*B[1] - 1.0175*B[4], "July Balance Constraint")
question_2_model.addConstr(w[3] == w[2] + 580000 - 200000 - 1.0275*B[0] - 1.0225*B[3] - 1.0175*B[5], "August Balance Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [84]:
# Add the cash flow constraints
May_Cash_Flow_Constraint = question_2_model.addConstr(w[0] >= 25000, "May Cash Flow Constraint")
June_Cash_Flow_Constraint = question_2_model.addConstr(w[1] >= 20000, "June Cash Flow Constraint")
July_Cash_Flow_Constraint = question_2_model.addConstr(w[2] >= 35000, "July Cash Flow Constraint")
August_Cash_Flow_Constraint = question_2_model.addConstr(w[3] >= 18000, "August Cash Flow Constraint")

In [85]:
# Add the borrowing constraints
question_2_model.addConstr((B[0] + B[1] + B[2]) <= 250000, "May Borrowing Constraint")
question_2_model.addConstr((B[3] + B[4]) <= 150000, "June Borrowing Constraint")
question_2_model.addConstr(B[5] <= 350000, "July Borrowing Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [86]:
# Ratio constraint
question_2_model.addConstr((0.65*(w[0] + w[1])) <= w[2], name="Ratio constraint")

<gurobi.Constr *Awaiting Model Update*>

In [87]:
# Optimally solve the problem
question_2_model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 12 rows, 10 columns and 32 nonzeros
Model fingerprint: 0xcc017ff0
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]
Presolve removed 5 rows and 0 columns
Presolve time: 0.00s
Presolved: 7 rows, 10 columns, 27 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+30   1.000000e+30   8.000000e+00      0s
       6    3.2709527e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.270952703e+05


In [88]:
# The status of the model (Optimization Status Codes)
print("Model Status: ", question_2_model.status)

Model Status:  2


In [89]:
# Number of variables in the model
print("Number of Decision Variables: ", question_2_model.numVars)

Number of Decision Variables:  10


In [90]:
# Value of the objective function
print("Total Amount of Money: ", round(question_2_model.objVal, 2))

Total Amount of Money:  327095.27


In [91]:
# Print the decision variables
print(question_2_model.printAttr('X'))


    Variable            X 
-------------------------
   Borrow[0]         5000 
   Borrow[3]      80945.9 
   Borrow[4]      54054.1 
   Wealth[0]        25000 
   Wealth[1]        20000 
   Wealth[2]        35000 
   Wealth[3]       327095 
None


a) The answer is 6

b) June cash balance >= $20,000

c) The formula: w[2] / (w[0] + w[1]) <= 0.65 or 65%
Written out, July cash balance / (May cash balance + June cash balance) <= 0.65 or 65%

To make it linear: 0.65*(w[0] + w[1]) <= w[2]
Written out, 0.65*(May cash balance + June cash balance) <= July cash balance

d)

In [92]:
# Get the optimal values of decision variables
borrowing_money = question_2_model.getAttr('x', B)

# Display the optimal values
print("Optimal Borrowing Values:")
for i in range(len(borrowing_money)):
    print(f"Borrow[{i}] = {borrowing_money[i]}")

# Calculate the total amount borrowed
total_borrowing_amount = borrowing_money[0] + borrowing_money[1] + borrowing_money[2] + borrowing_money[3] + borrowing_money[4] + borrowing_money[5]
print("\nTotal Borrowing Amount:", total_borrowing_amount)
total_repay_amount = 1.0275*borrowing_money[0] + 1.0225*borrowing_money[1] + 1.0175*borrowing_money[2] + 1.0225*borrowing_money[3] + 1.0175*borrowing_money[4] + 1.0175*borrowing_money[5]
print("\nTotal Repayment Amount:", total_repay_amount)


Optimal Borrowing Values:
Borrow[0] = 5000.0
Borrow[1] = 0.0
Borrow[2] = 0.0
Borrow[3] = 80945.94594594595
Borrow[4] = 54054.05405405405
Borrow[5] = 0.0

Total Borrowing Amount: 140000.0

Total Repayment Amount: 142904.72972972973


e) Looking above, the answer is $5,000

f) $327,095.27

g)

In [93]:
# Print sensitivity information
print("")
print(f"Sensitivity Information for June Cash Flow Constraint {June_Cash_Flow_Constraint.pi:.2f}:")
print("(LHS, RHS, Slack): ", (question_2_model.getRow(June_Cash_Flow_Constraint).getValue(), June_Cash_Flow_Constraint.RHS, June_Cash_Flow_Constraint.slack))
print("Shadow Price: ", June_Cash_Flow_Constraint.pi)
print("Range of Feasibility: ", (June_Cash_Flow_Constraint.SARHSUp, June_Cash_Flow_Constraint.SARHSLow))



Sensitivity Information for June Cash Flow Constraint -0.02:
(LHS, RHS, Slack):  (20000.0, 20000.0, 0.0)
Shadow Price:  -0.01758599508599512
Range of Feasibility:  (28846.153846153844, -0.0)


In [94]:
# Check if the optimization was successful
if question_2_model.status == gb.GRB.OPTIMAL:
    # Print the sensitivity analysis for the amount sold
    print("Optimal Amount Sold:")
    print(f"{'1'} = {B[0].x, B[0].RC, B[0].SAObjUp, B[0].SAObjLow}")
    print(f"{'2'} = {B[1].x, B[1].RC, B[1].SAObjUp, B[1].SAObjLow}")
    print(f"{'3'} = {B[2].x, B[2].RC, B[2].SAObjUp, B[2].SAObjLow}")
    print(f"{'4'} = {B[3].x, B[3].RC, B[3].SAObjUp, B[3].SAObjLow}")
    print(f"{'5'} = {B[4].x, B[4].RC, B[4].SAObjUp, B[4].SAObjLow}")
    print(f"{'6'} = {B[5].x, B[5].RC, B[5].SAObjUp, B[5].SAObjLow}")
else:
    print("Optimization was not successful.")

Optimal Amount Sold:
1 = (5000.0, 0.0, 0.0050000000000001155, -2.4570024569836973e-05)
2 = (0.0, -2.4570024569836973e-05, 2.4570024569836973e-05, -inf)
3 = (0.0, -0.012893749999999957, 0.012893749999999957, -inf)
4 = (80945.94594594595, 0.0, 2.444987775042457e-05, -0.0050000000000001155)
5 = (54054.05405405405, 0.0, 0.012806250000000146, -2.444987775042457e-05)
6 = (0.0, -0.012585995085995227, 0.012585995085995227, -inf)


In [95]:
# Change in Objective Function Value
change_in_ofv = June_Cash_Flow_Constraint.pi * (27500 - June_Cash_Flow_Constraint.RHS)

# Additional money needed to be borrowed
additional_borrowing = -change_in_ofv  # Note: Multiply by -1 to make it positive

print("Change in Objective Function Value:", change_in_ofv)
print("Additional Money Needed to be Borrowed:", additional_borrowing)

Change in Objective Function Value: -131.8949631449634
Additional Money Needed to be Borrowed: 131.8949631449634


In [96]:
# Create the optimization model
question_2_model2 = gb.Model("Question 2: Sunnyshore Bay")

# Create the three classes of decision variables where each Python
# variable represents a different number of Gurobi decision variables
B = question_2_model2.addVars(6, lb=0, vtype=GRB.CONTINUOUS, name="Borrow")
w = question_2_model2.addVars(4, lb=0, vtype=GRB.CONTINUOUS, name="Wealth")

# The objective function
question_2_model2.setObjective(w[3], GRB.MAXIMIZE)

# Add the balance constraints
question_2_model2.addConstr(w[0] == 140000 + 180000 - 300000 + B[0] + B[1] + B[2], "May Balance Constraint")
question_2_model2.addConstr(w[1] == w[0] + 260000 - 400000 + B[3] + B[4] - 1.0175*B[2], "June Balance Constraint")
question_2_model2.addConstr(w[2] == w[1] + 420000 - 350000 + B[5] - 1.0225*B[1] - 1.0175*B[4], "July Balance Constraint")
question_2_model2.addConstr(w[3] == w[2] + 580000 - 200000 - 1.0275*B[0] - 1.0225*B[3] - 1.0175*B[5], "August Balance Constraint")

# Add the cash flow constraints
May_Cash_Flow_Constraint = question_2_model2.addConstr(w[0] >= 25000, "May Cash Flow Constraint")
June_Cash_Flow_Constraint = question_2_model2.addConstr(w[1] >= 27500, "June Cash Flow Constraint")
July_Cash_Flow_Constraint = question_2_model2.addConstr(w[2] >= 35000, "July Cash Flow Constraint")
August_Cash_Flow_Constraint = question_2_model2.addConstr(w[3] >= 18000, "August Cash Flow Constraint")

# Add the borrowing constraints
question_2_model2.addConstr((B[0] + B[1] + B[2]) <= 250000, "May Borrowing Constraint")
question_2_model2.addConstr((B[3] + B[4]) <= 150000, "June Borrowing Constraint")
question_2_model2.addConstr(B[5] <= 350000, "July Borrowing Constraint")

# Ratio constraint
question_2_model2.addConstr((0.65*(w[0] + w[1])) <= w[2], name="Ratio constraint")

# Optimally solve the problem
question_2_model2.optimize()

# Value of the objective function
print("Total Amount of Money: ", round(question_2_model2.objVal, 2))


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 12 rows, 10 columns and 32 nonzeros
Model fingerprint: 0x502c54de
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]
Presolve removed 5 rows and 0 columns
Presolve time: 0.00s
Presolved: 7 rows, 10 columns, 27 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+30   1.000000e+30   8.000000e+00      0s
       6    3.2696338e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.269633753e+05
Total Amount of Money:  326963.38


In [97]:
# Get the optimal values of decision variables
borrowing_money = question_2_model2.getAttr('x', B)

# Display the optimal values
print("Optimal Borrowing Values:")
for i in range(len(borrowing_money)):
    print(f"Borrow[{i}] = {borrowing_money[i]}")

# Calculate the total amount borrowed
total_borrowing_amount = borrowing_money[0] + borrowing_money[1] + borrowing_money[2] + borrowing_money[3] + borrowing_money[4] + borrowing_money[5]
print("\nTotal Borrowing Amount:", total_borrowing_amount)
total_repay_amount = 1.0275*borrowing_money[0] + 1.0225*borrowing_money[1] + 1.0175*borrowing_money[2] + 1.0225*borrowing_money[3] + 1.0175*borrowing_money[4] + 1.0175*borrowing_money[5]
print("\nTotal Repayment Amount:", total_repay_amount)

Optimal Borrowing Values:
Borrow[0] = 5000.0
Borrow[1] = 0.0
Borrow[2] = 0.0
Borrow[3] = 81074.93857493858
Borrow[4] = 61425.06142506142
Borrow[5] = 0.0

Total Borrowing Amount: 147500.0

Total Repayment Amount: 150536.6246928747


h)

In [98]:
# Create the optimization model for the dual problem
dual_model = gb.Model("Dual of Question 2: Sunnyshore Bay")

# Create dual variables for each constraint in the primal model
y_may_balance = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="May_Balance_Dual")
y_june_balance = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="June_Balance_Dual")
y_july_balance = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="July_Balance_Dual")
y_august_balance = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="August_Balance_Dual")
y_may_cash_flow = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="May_Cash_Flow_Dual")
y_june_cash_flow = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="June_Cash_Flow_Dual")
y_july_cash_flow = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="July_Cash_Flow_Dual")
y_august_cash_flow = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="August_Cash_Flow_Dual")
y_may_borrowing = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="May_Borrowing_Dual")
y_june_borrowing = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="June_Borrowing_Dual")
y_july_borrowing = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="July_Borrowing_Dual")
y_ratio_constraint = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Ratio_Constraint_Dual")

# Set the objective function for the dual problem (minimize)
dual_model.setObjective(25000*y_may_cash_flow + 20000*y_june_cash_flow + 35000*y_july_cash_flow + 18000*y_august_cash_flow
                        + 250000*y_may_borrowing + 150000*y_june_borrowing + 350000*y_july_borrowing
                        + (0.65 * (y_may_balance + y_june_balance)), GRB.MINIMIZE)

# Add dual constraints for each primal variable
dual_model.addConstr(y_may_balance >= 1, "May_Balance_Dual_Constraint")
dual_model.addConstr(y_june_balance >= 1.0175, "June_Balance_Dual_Constraint")
dual_model.addConstr(y_july_balance >= 1.0225, "July_Balance_Dual_Constraint")
dual_model.addConstr(y_august_balance >= 1.0275, "August_Balance_Dual_Constraint")

dual_model.addConstr(y_may_cash_flow >= 0, "May_Cash_Flow_Dual_Constraint")
dual_model.addConstr(y_june_cash_flow >= 0, "June_Cash_Flow_Dual_Constraint")
dual_model.addConstr(y_july_cash_flow >= 0, "July_Cash_Flow_Dual_Constraint")
dual_model.addConstr(y_august_cash_flow >= 0, "August_Cash_Flow_Dual_Constraint")

dual_model.addConstr(y_may_borrowing >= 0, "May_Borrowing_Dual_Constraint")
dual_model.addConstr(y_june_borrowing >= 0, "June_Borrowing_Dual_Constraint")
dual_model.addConstr(y_july_borrowing >= 0, "July_Borrowing_Dual_Constraint")

dual_model.addConstr(y_ratio_constraint >= 0.65, "Ratio_Constraint_Dual_Constraint")

# Optimally solve the dual problem
dual_model.optimize()

# The status of the model (Optimization Status Codes)
print("Dual Model Status: ", dual_model.status)

# Number of variables in the model
print("Number of Dual Variables: ", dual_model.numVars)

# Value of the objective function
print("Dual Objective Value: ", round(dual_model.objVal, 2))

# Print the dual variables
print(dual_model.printAttr('X'))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 12 rows, 12 columns and 12 nonzeros
Model fingerprint: 0xc51e8bfc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-01, 4e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-01, 1e+00]
Presolve removed 12 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3113750e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.311375000e+00
Dual Model Status:  2
Number of Dual Variables:  12
Dual Objective Value:  1.31

    Variable            X 
-------------------------
May_Balance_Dual            1 
June_Balance_Dual       1.0175 
July_Balance_Dual       1.0225 
August_Balance_Dual

In [99]:
# Create the optimization model for the dual problem
dual_model = gb.Model("Dual of Question 2: Sunnyshore Bay")

# Create dual variables for each constraint in the primal model
y_may_balance = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="May_Balance_Dual")
y_june_balance = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="June_Balance_Dual")
y_july_balance = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="July_Balance_Dual")
y_august_balance = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="August_Balance_Dual")
y_may_cash_flow = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="May_Cash_Flow_Dual")
y_june_cash_flow = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="June_Cash_Flow_Dual")
y_july_cash_flow = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="July_Cash_Flow_Dual")
y_august_cash_flow = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="August_Cash_Flow_Dual")
y_may_borrowing = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="May_Borrowing_Dual")
y_june_borrowing = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="June_Borrowing_Dual")
y_july_borrowing = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="July_Borrowing_Dual")
y_ratio_constraint = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Ratio_Constraint_Dual")

# Set the objective function for the dual problem (minimize)
dual_model.setObjective(
    140000*y_may_balance + 260000*y_june_balance + 420000*y_july_balance + 580000*y_august_balance
    + 25000*y_may_cash_flow + 20000*y_june_cash_flow + 35000*y_july_cash_flow + 18000*y_august_cash_flow
    + 250000*y_may_borrowing + 150000*y_june_borrowing + 350000*y_july_borrowing
    + 0.65*y_ratio_constraint, GRB.MINIMIZE
)

# Add dual constraints for each primal variable
dual_model.addConstr(y_may_balance >= 1, "May_Balance_Dual_Constraint")
dual_model.addConstr(y_june_balance >= 1.0175, "June_Balance_Dual_Constraint")
dual_model.addConstr(y_july_balance >= 1.0225, "July_Balance_Dual_Constraint")
dual_model.addConstr(y_august_balance >= 1.0275, "August_Balance_Dual_Constraint")

dual_model.addConstr(y_may_cash_flow >= 0, "May_Cash_Flow_Dual_Constraint")
dual_model.addConstr(y_june_cash_flow >= 0, "June_Cash_Flow_Dual_Constraint")
dual_model.addConstr(y_july_cash_flow >= 0, "July_Cash_Flow_Dual_Constraint")
dual_model.addConstr(y_august_cash_flow >= 0, "August_Cash_Flow_Dual_Constraint")

dual_model.addConstr(y_may_borrowing >= 0, "May_Borrowing_Dual_Constraint")
dual_model.addConstr(y_june_borrowing >= 0, "June_Borrowing_Dual_Constraint")
dual_model.addConstr(y_july_borrowing >= 0, "July_Borrowing_Dual_Constraint")

dual_model.addConstr(y_ratio_constraint >= 0.65, "Ratio_Constraint_Dual_Constraint")

# Optimally solve the dual problem
dual_model.optimize()

# The status of the model (Optimization Status Codes)
print("Dual Model Status: ", dual_model.status)

# Number of variables in the model
print("Number of Dual Variables: ", dual_model.numVars)

# Value of the objective function
print("Dual Objective Value: ", round(dual_model.objVal, 2))

# Print the dual variables

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D60)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 12 rows, 12 columns and 12 nonzeros
Model fingerprint: 0x1e460044
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-01, 6e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-01, 1e+00]
Presolve removed 12 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4299504e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.429950423e+06
Dual Model Status:  2
Number of Dual Variables:  12
Dual Objective Value:  1429950.42
